In [ ]:
from obspy.clients.fdsn import Client
client = Client("iris")
print(client)

In [ ]:
client.services

In [ ]:
import requests
url='https://service.iris.edu/irisws/fedcatalog/1/'
r=requests.get(url + "query", params={"net":"A*","sta":"OK*","cha":"*HZ"}, verify=False)


In [137]:
for p in r.iter_lines():
    if p.startswith("DATACENTER"):
        print(p)

DATACENTER=GEOFON,http://geofon.gfz-potsdam.de
DATACENTER=INGV,http://www.ingv.it
DATACENTER=IRISDMC,http://ds.iris.edu
DATACENTER=NCEDC,http://www.ncedc.org
DATACENTER=ORFEUS,http://www.orfeus-eu.org
DATACENTER=SCEDC,http://scedc.caltech.edu/


In [ ]:
rtext= '''
    sta=A*
    minlat=34
    maxlat=38
    cha=?HZ
    starttime=2015-01-01
    endtime=2016-01-02'''
r = requests.post(url+"query",data='includeoverlaps=true\nA* OR* * BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00', verify=False)
print(r)
print(r.content)

In [5]:
class RequestLine(object):
    def __init__(self, line):
        self.line = line.strip()
        
    def is_empty(self):
        return self.line == ""

    def is_datacenter(self):
        return self.line.startswith('DATACENTER=')

    def is_param(self):
        # true for datacenter, services, and parameter_list
        return '=' in self.line

    def is_request(self):
        return len(self.line.split()) == 6 # and test field values?

    def is_service(self):
        # parse param_name
        return self.is_param() and self.line.split("=")[0].isupper() and not self.is_datacenter()

    def __repr__(self):
        return self.line

    def __str__(self):
        return self.line


In [34]:

class ParserState(object):
    '''
    parser states: PREPARSE, PARAMLIST, EMPTY_LINE, DATACENTER, SERVICE, REQUEST, DONE
    PREPARSE -> [PARAMLIST | EMPTY_LINE | DATACENTER]
    PARAMLIST -> [PARAMLIST | EMPTY_LINE]
    EMPTY_LINE -> [EMPTY_LINE | DATACENTER | DONE]
    DATACENTER -> [SERVICE]
    SERVICE -> [SERVICE | REQUEST]
    REQUEST -> [REQUEST | EMPTY_LINE | DONE ]
    '''

    @staticmethod
    def parse(line, this_response):
        '''abstract'''
        raise NotImplementedError, "ParserState.parse()"

    @staticmethod
    def next(line):
        '''abstract'''
        raise NotImplementedError, "ParserState.next()"

class PreParse(ParserState):
    '''Initial ParserState'''

    @staticmethod
    def parse(line, this_response):
        return this_response

    @staticmethod
    def next(line):
        if line.is_empty():
            return EmptyItem #EMPTY_LINE
        elif line.is_datacenter():
            return DatacenterItem #DATACENTER
        elif line.is_param():
            return ParameterItem #PARAMLIST
        else:
            return ParserState

class ParameterItem(ParserState):
    '''handle a parameter'''

    @staticmethod
    def parse(line, this_response):
        '''Parse: param=value'''
        this_response.add_common_parameters(line)
        return this_response

    @staticmethod
    def next(line):
        if line.is_empty():
            return EmptyItem() #EMPTY_LINE
        elif line.is_param():
            return self
        else:
            raise RuntimeError, "Parameter should be followed by another parameter or an empty line"

class EmptyItem(ParserState):
    '''handle an empty line'''

    @staticmethod
    def parse(line, this_response):
        return this_response
    
    @staticmethod
    def next(line):
        if line.is_empty():
            return EmptyItem #no state change
        elif line.is_datacenter():
            return DatacenterItem #DATACENTER
        else:
            raise RuntimeError, "expected either a DATACENTER or another empty line"

class DatacenterItem(ParserState):
    '''handle data center'''

    @staticmethod
    def parse(line, this_response):
        '''Parse: DATACENTER=id,http://url...'''
        _, rest = str(line).split('=')
        active_id, url = rest.split(',')
        this_response = new_federated_response(active_id)
        print("new response", this_response)
        this_response.add_service("DATACENTER", url)
        return this_response

    @staticmethod
    def next(line):
        if line.is_service():
            return ServiceItem
        else:
            raise RuntimeError, "DATACENTER line should be followed by a service"

class ServiceItem(ParserState):
    '''handle service description'''

    @staticmethod
    def parse(line, this_response):
        '''Parse: SERICENAME=http://service.url/'''
        svc_name, url = str(line).split('=')
        this_response.add_service(svc_name, url)
        return this_response

    @staticmethod
    def next(line):
        if line.is_service():
            return ServiceItem
        elif line.is_request():
            return RequestItem
        else:
            raise RuntimeError, "Service desc. should be followed by a request or another service"

class RequestItem(ParserState):
    '''handle request lines'''

    @staticmethod
    def parse(line, this_response):
        '''Parse: NT STA LC CHA YYYY-MM-DDThh:mm:ss YY-MM-DDThh:mm:ss'''
        this_response.add_request_lines(line)
        return this_response

    @staticmethod
    def next(line):
        if line.is_request():
            return RequestItem
        elif line.is_empty():
            return EmptyItem
        else:
            raise RuntimeError, "Requests should be followed by another request or an empty line"

In [54]:

class FederatedResponse(object):
    '''
    >>> fed_resp = FederatedResponse("IRISDMC")
    >>> fed_resp.add_common_parameters(["lat=50","lon=20","level=cha"])
    >>> fed_resp.add_service("STATIONSERVICE","http://service.iris.edu/fdsnws/station/1/")
    >>> fed_resp.add_request_line("AI ORCD -- BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00")
    >>> fed_resp.add_request_line("AI ORCD 04 BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00")
    >>> print(fed_resp.request_text("STATIONSERVICE"))

    level=cha
    AI ORCD -- BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00
    AI ORCD 04 BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00
    '''

    ok_parameters = {"DATASELECTSERVICE":["longestonly"],
                        "STATIONSERVICE":["level"]}

    def __init__(self, datacenter_id):
        self.datacenter_id = datacenter_id
        self.common_parameters = []
        self.services = {}
        self.request_lines = []

    def add_service(self, service_name, service_url):
        self.services[service_name] = service_url

    def add_common_parameters(self, common_parameters):
        if isinstance(common_parameters, str):
            self.common_parameters.append(common_parameters)
        elif isinstance(common_parameters, RequestLine):
            self.common_parameters.append(str(common_parameters))
        else:
            self.common_parameters.extend(common_parameters)

    def add_request_lines(self, request_lines):
        if isinstance(request_lines, str):
            self.request_lines.append(request_lines)
        elif isinstance(request_lines, RequestLine):
            self.request_lines.append(str(request_lines))
        else:
            self.request_lines.extend(request_lines)

    def add_request_line(self, request_line):
        self.request_lines.append(request_line)

    def request_text(self, target_service):
        reply = self.selected_common_parameters(target_service)
        reply.extend(self.request_lines)
        return "\n".join(reply)

    def selected_common_parameters(self, target_service):
        reply = []
        for good in FederatedResponse.ok_parameters[target_service]:
            reply.extend([c for c in self.common_parameters if c.startswith(good + "=")])
        return reply
    
    def __repr__(self):
        return self.datacenter_id + "\n" + self.request_text("STATIONSERVICE")

In [65]:
class StreamingFederatedResponseParser(object):
    '''Iterate through stream, returning FederatedResponse objects for each datacenter'''
    def __init__(self, stream_iterator):
        self.stream_iterator = stream_iterator() # stream_iterator feeds us line by line
        self.state = PreParse
        self.n_datacenters = 0
        self.curr_datacenter = FederatedResponse("PRE_CENTER")
        self.common_params = None
        self.line = None

    def __iter__(self):
        return self

    def next(self):
        request_was_processed = False
        if self.line is not None:
            self.curr_datacenter = self.state.parse(self.line, self.curr_datacenter) #left before processing
            self.line = None

        for self.line in self.stream_iterator:
            self.line = RequestLine(self.line)
            self.state = self.state.next(self.line)
            if request_was_processed and (self.state is not RequestItem):
                self.curr_datacenter.common_parameters = self.common_params
                return self.curr_datacenter
            if self.state == DatacenterItem and self.curr_datacenter.datacenter_id == "PRE_CENTER":
                self.common_params = self.curr_datacenter.common_parameters
            self.curr_datacenter = self.state.parse(self.line, self.curr_datacenter)
            if self.state == RequestItem:
                request_was_processed = True
        raise StopIteration

    __next__ = next
    
def new_federated_response(ds_id):
    return FederatedResponse(ds_id)

In [70]:
import requests
url='https://service.iris.edu/irisws/fedcatalog/1/'
r=requests.get(url + "query", params={"net":"A*","sta":"*","cha":"*","includeoverlaps":"true","level":"sta"}, verify=False)
frp = StreamingFederatedResponseParser(r.iter_lines)
print(r.content)
for n in frp:
    #print(n)
    print(n.request_text("STATIONSERVICE"))


/Users/celso/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


level=sta

DATACENTER=GEOFON,http://geofon.gfz-potsdam.de
DATASELECTSERVICE=http://geofon.gfz-potsdam.de/fdsnws/dataselect/1/
STATIONSERVICE=http://geofon.gfz-potsdam.de/fdsnws/station/1/
AF BLWY -- BHE 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- BHN 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- BHZ 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- HHE 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- HHN 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- HHZ 2013-03-06T00:00:00 2599-12-31T23:59:59
AF CER -- BHE 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- BHN 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- BHZ 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- HHE 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- HHN 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- HHZ 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- LHE 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- LHN 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- LHZ 2007-03-15T00:47:00 2599-12-31T23:59

In [9]:
fed_resp = FederatedResponse("IRISDMC")
fed_resp.add_common_parameters("dummy=dummy")
fed_resp.add_common_parameters(["lat=50","lon=20","level=cha"])
fed_resp.add_service("STATIONSERVICE","http://service.iris.edu/fdsnws/station/1/")
fed_resp.add_request_lines("AI ORCD -- BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00")
fed_resp.add_request_lines(["AI ORCD 04 BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00"])
print(fed_resp.request_text("STATIONSERVICE"))

level=cha
AI ORCD -- BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00
AI ORCD 04 BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00


In [ ]:
frp = FederatedResponseParser(r.iter_lines)
print("frp" , frp.__class__)

In [52]:

def parse_federated_response(block_text):
    '''create a list of FederatedResponse objects, one for each datacenter in response'''
    fed_resp = []
    current_datacenter = FederatedResponse("PRE_CENTER")
    common_params = None
    state = PreParse
    for line in block_text.splitlines():
        req_line = RequestLine(line)
        state = state.next(req_line)
        if state == DatacenterItem:
            if current_datacenter.datacenter_id=="PRE_CENTER":
                print("copying over")
                for x in current_datacenter.common_parameters:
                    print(x)
                common_params= current_datacenter.common_parameters
            current_datacenter = state.parse(req_line, current_datacenter)
            current_datacenter.common_parameters = common_params
            fed_resp.append(current_datacenter)
        else:
            current_datacenter = state.parse(req_line, current_datacenter)
    if not fed_resp[-1].request_lines:
        del fed_resp[-1]
    return fed_resp

In [71]:
a = parse_federated_response(r.content)
for z in a:
    print(z.request_text("STATIONSERVICE"))
z.datacenter_id

copying over
level=sta
('new response', GEOFON
)
('new response', INGV
)
('new response', IRISDMC
)
('new response', NCEDC
)
('new response', ORFEUS
)
('new response', SCEDC
)
level=sta
AF BLWY -- BHE 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- BHN 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- BHZ 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- HHE 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- HHN 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- HHZ 2013-03-06T00:00:00 2599-12-31T23:59:59
AF CER -- BHE 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- BHN 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- BHZ 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- HHE 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- HHN 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- HHZ 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- LHE 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- LHN 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- LHZ 2007-03-15T00:47:00 2599-12-31T23:59:5

In [40]:
a[0].common_parameters

[]

In [136]:
print(r.content)

level=sta

DATACENTER=GEOFON,http://geofon.gfz-potsdam.de
DATASELECTSERVICE=http://geofon.gfz-potsdam.de/fdsnws/dataselect/1/
STATIONSERVICE=http://geofon.gfz-potsdam.de/fdsnws/station/1/
AF BLWY -- BHE 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- BHN 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- BHZ 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- HHE 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- HHN 2013-03-06T00:00:00 2599-12-31T23:59:59
AF BLWY -- HHZ 2013-03-06T00:00:00 2599-12-31T23:59:59
AF CER -- BHE 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- BHN 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- BHZ 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- HHE 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- HHN 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- HHZ 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- LHE 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- LHN 2007-03-15T00:47:00 2599-12-31T23:59:59
AF CER -- LHZ 2007-03-15T00:47:00 2599-12-31T23:59

In [143]:
from obspy.clients.fdsn import Client
def fed_sta(**kwarg):
    remap = {"IRISDMC":"IRIS", "GEOFON":"GFZ", "SED":"ETH", "USPSC":"USP"}
    # need to add to URL_MAPPINGS!
    allInv=[]
    url='https://service.iris.edu/irisws/fedcatalog/1/'
    r=requests.get(url + "query", params=kwarg, verify=False)
    print "asking from..."
    for p in r.iter_lines():
        if p.startswith("DATACENTER"):
            print(p)
        
    sfrp = StreamingFederatedResponseParser(r.iter_lines)
    for datac in sfrp:
        dc_id = datac.datacenter_id
        if dc_id in remap:
            dc_id = remap[dc_id]
        client = Client(dc_id)
        #requests that are too large could be denied (#413) and need to be chunked.
        print(datac.request_text("STATIONSERVICE").count('\n'))
        try:
            inv = client.get_stations_bulk(bulk=datac.request_text("STATIONSERVICE"))
            if not allInv:
                allInv = inv
            else:
                allInv += inv
        except:
            print(dc_id, "error!")
            
    return allInv

In [144]:
inv=fed_sta(network="*", station="A*", channel="BHZ", level="sta", includeoverlaps=True )
print(inv)

/Users/celso/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


asking from...
DATACENTER=BGR,http://eida.bgr.de
DATACENTER=GEOFON,http://geofon.gfz-potsdam.de
DATACENTER=INGV,http://www.ingv.it
DATACENTER=IPGP,http://centrededonnees.ipgp.fr
DATACENTER=IRISDMC,http://ds.iris.edu
DATACENTER=KOERI,http://www.koeri.boun.edu.tr/2/tr
DATACENTER=NCEDC,http://www.ncedc.org
DATACENTER=NIEP,http://www.orfeus-eu.org/eida/eida_niep.html
DATACENTER=ORFEUS,http://www.orfeus-eu.org
DATACENTER=RESIF,http://www.resif.fr
DATACENTER=SCEDC,http://scedc.caltech.edu/
DATACENTER=SED,http://www.seismo.ethz.ch
DATACENTER=USPSC,http://www.moho.iag.usp.br
('new response', BGR
)
7
('new response', GEOFON
)
113
('new response', INGV
)
40
('INGV', 'error!')
('new response', IPGP
)
9
('new response', IRISDMC
)
946
('new response', KOERI
)
11
('new response', NCEDC
)
3
('new response', NIEP
)
1
('new response', ORFEUS
)
41
('new response', RESIF
)
48
('RESIF', 'error!')
('new response', SCEDC
)
18
('new response', SED
)
35
('new response', USPSC
)
15
Inventory created at 2017-04

In [145]:
print(inv)

Inventory created at 2017-04-01T15:47:00.215775Z
	Created by: ObsPy 1.0.2
		    https://www.obspy.org
	Sending institution: IRIS-DMC,NCEDC,SCEDC,SeisComP3 (BGR,GFZ,IPGP,IRIS-DMC,KOERI,NCEDC,NIEP,ODC,SCEDC,SED)
	Contains:
		Networks (181):
			GR
			TH
			2F
			DK
			EE
			GE
			HT
			IA
			IS
			JS
			KC
			NU
			RO
			WM
			XE
			ZD
			ZD
			ZF
			ZP
			ZZ
			G
			WI
			1E
			7A
			AF
			AK
			AT
			AU
			AV
			AZ
			BL
			C1
			CC
			CI
			CU
			DK
			DW
			G
			GE
			GS
			GY
			HL
			II
			IM
			IU
			JP
			KN
			KR
			KZ
			LD
			MC
			MI
			MN
			NB
			NJ
			NU
			OE
			OO
			PR
			RV
			S
			SP
			TA
			TW
			UK
			US
			WI
			X1
			X4
			X9
			XA
			XA
			XB
			XB
			XC
			XC
			XC
			XD
			XD
			XE
			XF
			XG
			XG
			XH
			XI
			XI
			XI
			XJ
			XJ
			XJ
			XK
			XL
			XM
			XM
			XN
			XN
			XN
			XO
			XP
			XQ
			XS
			XT
			XT
			XV
			XW
			XW
			XW
			XW
			XW
			XW
			XW
			XW
			XY
			XZ
			Y3
			YB
			YB
			YC
			YD
			YF
			YI
			YI
			YJ
			YJ
			YJ
			YL
			YL
		